In [1]:
from ast import literal_eval
import pandas as pd 
from copy import deepcopy
from shapely.geometry import LinearRing, Polygon
import shapely

In [2]:
df = pd.read_csv(r'C:\Users\roberto.delprete\OneDrive - ESA\Desktop\PyScripts\ESA\pyraws\pyraws\database\thraws_db.csv', index_col=[0])
df

,Unnamed: 0,ID_event,Start,End,Sat,"Coords (Lat, Lon)",class,Raw_useful_granules,Raw_complementary_granules,Polygon,Raw_files,l1c_files,bbox_list
0,Barren_Island_00,28/09/2018,28/09/2018 23:59,S2A,"(12.28474241, 93.86212046)",eruption,[2],[4],POLYGON ((93.82076157486179 12.244061101857305...,['S2B_OPER_MSI_L0__GR_SGS__20170928T092515_S20...,['S2A_MSIL1C_20180928T040541_N0206_R047_T46PEU...,"{2: [[[403, 290], [427, 284], [435, 316], [410...","{2: [[[403, 290], [427, 284], [435, 316], [410..."
1,Barren_Island_01,28/10/2018,28/10/2018 23:59,S2A,"(12.28474241, 93.86212046)",eruption,[2],[4],POLYGON ((93.82076157486179 12.244061101857305...,['S2B_OPER_MSI_L0__GR_MTI__20171028T093953_S20...,['S2A_MSIL1C_20181028T040851_N0206_R047_T46PEU...,"{2: [[[391, 59], [412, 54], [418, 82], [398, 8...","{2: [[[391, 59], [412, 54], [418, 82], [398, 8..."
2,Barren_Island_02,29/07/2021,29/07/2021 23:59,S2B,"(12.28474241, 93.86212046)",eruption,[2],[4],POLYGON ((93.82076157486179 12.244061101857305...,['S2A_OPER_MSI_L0__GR_VGS2_20200729T074807_S20...,['S2B_MSIL1C_20210729T040549_N0301_R047_T46PEU...,"{2: [[[406, 280], [410, 279], [411, 284], [407...","{2: [[[406, 280], [410, 279], [411, 284], [407..."
3,Barren_Island_10,16/01/2017,16/01/2017 23:59,S2A,"(12.28474241, 93.86212046)",eruption,[1],[3],POLYGON ((93.82076157486179 12.244061101857305...,['S2A_OPER_MSI_L0__GR_MTI__20160819T093155_S20...,['S2A_MSIL1C_20170116T041111_N0204_R047_T46PEU...,"{1: [[[297, 770], [302, 768], [303, 773], [298...","{1: [[[297, 770], [302, 768], [303, 773], [298..."
4,Chillan_Nevados_de_01,25/05/2021,25/05/2021 23:59,S2A,"(-36.86703697, -71.37834332)",eruption,[1],[2],POLYGON ((-71.42883743575479 -36.9075757257842...,['S2B_OPER_MSI_L0__GR_SGS__20200119T174535_S20...,['S2A_MSIL1C_20210525T142731_N0300_R053_T19HBV...,"{1: [[[204, 1077], [224, 1073], [231, 1099], [...","{1: [[[204, 1077], [224, 1073], [231, 1099], [..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,Greece_5,19/07/2019,19/07/2019 23:59,S2A,"(38.4886, 24.0979)",fire,"[2, 3, 5]","[4, 5, 3]",POLYGON ((24.046350885087246 38.44805006978415...,['S2B_OPER_MSI_L0__GR_SGS__20200214T112713_S20...,S2A_MSIL1C_20190705T090601_N0207_R050_T35SKC_2...,"{2: [[[36, 16], [54, 75], [110, 64], [92, 5], ...","{2: [[[(102, 81), (100, 74), (109, 73), (106, ..."
84,Kenya_0,19/02/2019,19/02/2019 23:59,S2A,"(-0.2038, 37.2755)",fire,"[0, 2, 3]","[2, 4, 5]",POLYGON ((37.23507544999801 -0.244496565730667...,['S2A_OPER_MSI_L0__GR_SGS__20180523T145924_S20...,S2A_MSIL1C_20190228T073841_N0207_R092_T37MBV_2...,"{0: [[[479, 241], [496, 301], [571, 286], [558...","{0: [[[(455, 268), (463, 283), (462, 284), (47..."
85,Latvia_1,18/08/2018,18/08/2018 23:59,S2A,"(57.9515, 24.8847)",fire,"[5, 6, 8, 9]","[3, None, 9, 8]","POLYGON ((24.80878827423171 57.91107379767839,...",['S2A_OPER_MSI_L0__GR_VGS1_20200906T201656_S20...,S2A_MSIL1C_20180801T095031_N0206_R079_T35VLE_2...,"{5: [[[1124, 132], [1147, 201], [1216, 174], [...","{5: [[[(1216, 179), (1256, 166), (1237, 113), ..."
86,Sweden_0,18/07/2018,18/07/2018 23:59,S2A,"(61.815, 14.4926)",fire,"[0, 2, 3]","[2, 4, 1]",POLYGON ((14.407348407846948 61.77459385719579...,['S2A_OPER_MSI_L0__GR_MPS__20181012T073622_S20...,S2A_MSIL1C_20180717T104021_N0206_R008_T33VVJ_2...,"{0: [[[3, 1042], [15, 1094], [54, 1068], [35, ...","{0: [[[(413, 1149), (429, 1141), (437, 1159), ..."


# Step 1: Create the function to buffer a bbox.

In [3]:
def checkNest(lista):
     return any(isinstance(i, list) for i in lista)


def checkMultiple(lista):
     '''Check if the granule contains multiple bbxoxes'''
     for item in lista:
          if checkNest(item):
               return True
     return False 


In [4]:
SF=1.15

def bufferCorrection(dictPoints: dict, scaleFactor: float=1.15):

    inputt = literal_eval(dictPoints)
    stored_output = {}
    # Processing:
    for key in inputt:
        container = inputt[key]

        if checkMultiple(container):
            stored_output[key] = []
            for idx, bbox in enumerate(container):
                points = bbox
                points.append(points[0])
                r = LinearRing(points)
                s = Polygon(r)
                bufferedPoly = shapely.affinity.scale(s, 
                                            xfact=SF, yfact=SF )
                xx,yy=bufferedPoly.exterior.coords.xy
                xx,yy=list(xx),list(yy)
                new_points = [(a,b) for a,b in zip(xx,yy)]
                stored_output[key].append(new_points)

                # if idx ==1:
                #     break

        
        else:
            points = container
            points.append(points[0])
            r = LinearRing(points)
            s = Polygon(r)
            bufferedPoly = shapely.affinity.scale(s, 
                                        xfact=SF, yfact=SF )
            xx,yy=bufferedPoly.exterior.coords.xy
            xx,yy=list(xx),list(yy)
            new_points = [(a,b) for a,b in zip(xx,yy)]
            stored_output[key] = new_points
    
    return stored_output


In [5]:
tmp['bbox_list_updated']=tmp['bbox_list'].apply(bufferCorrection)

NameError: name 'tmp' is not defined

In [ ]:
tmp

finalDB = pd.concat([tmp, notevent])
finalDB.reset_index(drop=True, inplace=True)

In [ ]:
finalDB.to_csv(r"C:\Users\roberto.delprete\OneDrive - ESA\Desktop\PyScripts\ESA\pyraws\pyraws\database\thraws_db_v2.csv")

# Step 2: Patch Study 

In [7]:
import csv, os
import sys
sys.path.insert(1, os.path.join("..", ".."))
sys.path.insert(1, os.path.join("..", "..", "scripts_and_studies", "hta_detection_algorithms"))
from pyraws.l0.l0_event import L0_event
from pyraws.l1.l1_event import L1C_event
from pyraws.l1.l1_tile import L1C_tile
from pyraws.utils.l0_utils import get_bands_shift
from pyraws.utils.l1_utils import read_L1C_image_from_tif
from pyraws.utils.database_utils import get_cfg_file_dict
from pyraws.utils.parse_csv_utils import parse_csv
from pyraws.utils.visualization_utils import plot_img1_vs_img2_bands
from s2pix_detector import s2pix_detector
from functools import partial
from geopy.distance import geodesic as GD
from geopy.geocoders import Nominatim
from geopy.point import Point
import geopandas
from geopandas import GeoSeries
import matplotlib.pyplot as plt
from matplotlib import patches
import numpy as np
from shapely.geometry import Polygon
from termcolor import colored
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import numpy as np
from copy import deepcopy
import pandas as pd
from ast import literal_eval
from IPython.display import clear_output
import time

# CONFIG_
os.environ['KMP_DUPLICATE_LIB_OK']='True'
if torch.cuda.is_available():
    device=torch.device("cuda")
else:
    device=torch.device("cpu")

plt.rcParams['figure.figsize'] = [10, 10]

%load_ext autoreload
%autoreload 2


def mulitBox(bbox_list):
    
     if any(isinstance(i, list) for i in bbox_list):
          return True

     return False



def sliding_window(image, window_size, overlap):
     """
     Generate sub-images of the input image using a sliding window.
     Parameters:
          - image (2D numpy array): The input image.
          - window_size (int): The size (in pixels) of the sliding window.
          - overlap (float): The degree of overlap between two consecutive window movements.
     
     Returns:
          - sub_images (dict): A dictionary with the top-left and bottom-right corner positions (x1, y1, x2, y2) as keys and the sub-images as values.
     """
     # Get the dimensions of the input image
     h, w = image.shape[:2]
     
     # Calculate the step size (in pixels) for the sliding window
     step_size = int(window_size * (1 - overlap))
     
     # Initialize the dictionary of sub-images
     sub_images = {}
     
     # Slide the window across the image
     for y in range(0, h, step_size):
          for x in range(0, w, step_size):
               # Extract the sub-image
               sub_image = image[y:y+window_size, x:x+window_size]
               klam = window_size - sub_image.shape[1]
               teta = window_size - sub_image.shape[0]
               sub_image = image[y-teta:y-teta+window_size, x-klam:x-klam+window_size]
               
               # Add the sub-image to the dictionary with the positions as the key
               sub_images[(x-klam, y-teta,  x-klam+window_size, y-teta+window_size)] = sub_image
     return sub_images



# Event config:
def find_granule_dim(event_name:str, key_granule):
     requested_bands=["B04","B8A", "B11","B12"]
     l0_event=L0_event(device=device)
     l0_event.from_database(event_name, requested_bands)
     _, stackable_couples=l0_event.stack_granules_couples()
     granule_list=list(set([val for sublist in stackable_couples for val in sublist]))
     
     l0_granule_registered=l0_event.coarse_coregistration([key_granule], use_complementary_granules=True, crop_empty_pixels=True, verbose=False)
     l0_granule_tensor=l0_granule_registered.as_tensor()[:,:,:]
     return [l0_granule_tensor.size(0),l0_granule_tensor.size(1)]



def pos2Shap(pos, offY):
     x1, y1, x2, y2 = pos
     # Only for calculating the polygon and the intersection.
     ShapP1 = (x1, offY-y1)
     ShapP2 = (x1, offY-y2)
     ShapP3 = (x2, offY-y2)
     ShapP4 = (x2, offY-y1)
     ShapP5 = ShapP1

     Shapatch = Polygon([ShapP1,ShapP2, ShapP3, ShapP4, ShapP5]).convex_hull
     return Shapatch

ModuleNotFoundError: No module named 'pyraws.l0.l0_event'

### Add the granule dimensions to the database.

In [ ]:
df = pd.read_csv(r'C:\Users\roberto.delprete\OneDrive - ESA\Desktop\PyScripts\ESA\pyraws\pyraws\database\thraws_db.csv')
df=df[df['ID_event']=='Etna_00'].bbox_list.item()
# df['Len']=[len(literal_eval(x)) for x in df['bbox_list'].to_list()]
# df = df[df['Len']>0]

# GranuleDim = []
# for idx, row in df.iterrows():
#      id_event = row.ID_event
#      print(idx, id_event)
#      try:
#           num_batches=row.Len
#           bbox_list = literal_eval(row.bbox_list)
#           id_event = row.ID_event
#           tmp = {}
#           for key_granule in bbox_list:
#                tmp[key_granule]=find_granule_dim(id_event, key_granule)
#           GranuleDim.append(tmp)
#           clear_output(wait=True)
          
#      except ValueError:
#           print("IS BROKEN:", row.ID_event)
#           break



# df['GranulesDim']=GranuleDim

In [ ]:
# CONFIG:
     # Requested bands:
requested_bands=["B04","B8A", "B11","B12"]
     # Save directory for later use.     
BS = r'C:\Users\roberto.delprete\OneDrive - ESA\Desktop\PyScripts\ESA\pyraws\scripts_and_studies\Test\patch_study_img'


for idx in range(len(df)):

     l0_event=L0_event(device=device)
     event_name = df.iloc[idx].ID_event
     print(f'Parsing event: {event_name}')

     key_granules = list(literal_eval(df.iloc[idx].bbox_list).keys())
     l0_event.from_database(event_name, requested_bands)
     _, stackable_couples=l0_event.stack_granules_couples()
     granule_list=list(set([val for sublist in stackable_couples for val in sublist]))
     for key_granule in key_granules:
          # Taking the L0 granule tensor:
          l0_granule_registered=l0_event.coarse_coregistration([key_granule], use_complementary_granules=True, crop_empty_pixels=True, verbose=False)
          l0_granule_tensor=l0_granule_registered.as_tensor()[:,:,:]
          img=l0_granule_tensor.numpy() # co-registered.
          offY = l0_granule_tensor.shape[0]

          # Taking the bbox_list:
          gbbox = literal_eval(df[df['ID_event']==event_name].bbox_list.item())[key_granule]
          if mulitBox(gbbox):
               BOXES = [[[x, offY-y] for [x, y] in tmp] for tmp in gbbox]
          else:
               gbbox = [[x, offY-y] for [x, y] in gbbox]
               BOXES = [gbbox]     

          # Bounding boxes of the events adjusted for shapely origin coordinates reference frame.
          POLYBOXES = [Polygon(x) for x in BOXES] # All the bounding boxes in shapley polygons here.

          # Splitting the images:
          sub_images = sliding_window(image=img, window_size=256, overlap=0.25) # Dict 
          for idy, pos in enumerate(sub_images):
               print('Starting from pos:',pos)
               A = sub_images[pos]
               # Only for calculating the polygon and the intersection.
               Shapatch = pos2Shap(pos, offY)
               BoxesIntersectPatch = [BOX.intersection(Shapatch).area for BOX in POLYBOXES]
               if max(BoxesIntersectPatch) > 0:
                    Area = max(BoxesIntersectPatch)
               else:
                    Area = 0

               print('Pos ', pos, ' Area = ', Area)
               Area = max(BoxesIntersectPatch)
               if Area > 5  :
                    Bfolder = 'EVENT'
               else:
                    Bfolder = 'BG'

               # SAVING:
               pd.to_pickle(filepath_or_buffer=f"{BS}\{Bfolder}\{event_name}_G{key_granule}_{str(pos)}_A{Area:.2f}.pkl", obj=A)
               tmp = A/A.max()
               fig, ax =  plt.subplots(figsize=(10,10))
               ax.imshow(tmp[:,:,1:])
               ax.set_axis_off()
               fig.tight_layout()
               fig.savefig(f"{BS}\{Bfolder}\{event_name}_G{key_granule}_QKL_{str(pos)}_A{Area:.2f}.jpg")
               clear_output(wait=True)


# TEST:

In [ ]:
l0_event=L0_event(device=device)
event_name = 'Nyamulagira_00'
Granulo = 4
print(f'Parsing event: {event_name}')
requested_bands=["B04","B8A", "B11","B12"]

l0_event.from_database(event_name, requested_bands)
_, stackable_couples=l0_event.stack_granules_couples()
granule_list=list(set([val for sublist in stackable_couples for val in sublist]))
print(granule_list)
l0_granule_registered=l0_event.coarse_coregistration([Granulo], use_complementary_granules=True, crop_empty_pixels=True, verbose=False)
l0_granule_tensor=l0_granule_registered.as_tensor()[:,:,:]
l0_granule_tensor = l0_granule_tensor / l0_granule_tensor.max()

x1, y1, x2, y2 = 192,896,448,1152

# Only for calculating the polygon and the intersection.
offY = l0_granule_tensor.shape[0]
ShapP1 = (x1, offY-y1)
ShapP2 = (x1, offY-y2)
ShapP3 = (x2, offY-y2)
ShapP4 = (x2, offY-y1)
ShapP5 = ShapP1
Shapatch = Polygon([ShapP1,ShapP2, ShapP3, ShapP4, ShapP5]).convex_hull
# For visualization:
P1 = (x1, y1)
P2 = (x1, y2)
P3 = (x2, y2)
P4 = (x2, y1)
P5 = P1
patchPoints = [P1, P2, P3, P4, P5]


fig, ax = plt.subplots(1,1, figsize=(10,10))
ax.imshow(l0_granule_tensor[:,:,1:])

gbbox = literal_eval(df[df['ID_event']==event_name].bbox_list.item())[Granulo]
if mulitBox(gbbox):
     BOXES = [[[x, offY-y] for [x, y] in tmp] for tmp in gbbox]
else:
     BOXES = [gbbox]

BOXES = [Polygon(t) for t in BOXES]
for bb in BOXES:
     bbb = patches.Polygon(bb.exterior.coords, linewidth=1, edgecolor='y', facecolor='none')
     ax.add_patch(bbb)

# SUBIMAGE PATCH
pBbox = patches.Polygon(patchPoints, linewidth=1, edgecolor='b', facecolor='none')
ax.add_patch(pBbox)
